<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## Capstone Project: Clustering Logistics Research Papers
[Part 1: Get Abstract](01_Get_Abstrast.ipynb) | **Part 2: Add Label** | [Part 3: EDA](03_EDA.ipynb) | [Part 4: Model Tuning](..)



### Introducion


This notebook focuses on labeling the topic category and keywords from abstracts by calling the Typhoon LLM API. It feeds the abstracts of files into the API to retrieve the corresponding answers from Typhoon LLM.

### Import Library

In [113]:
import pandas as pd
import numpy as np
import time
import os
from dotenv import load_dotenv
from openai import OpenAI

### Call Typhoon API

In [114]:
# Check get key
success = load_dotenv()
print(f"Load successful: {success}")

Load successful: True


In [115]:
# Check call API
client = OpenAI(
   api_key= os.getenv('API_KEY'),
   base_url='https://api.opentyphoon.ai/v1'
)

chat_completion = client.chat.completions.create(
    model="typhoon-v1.5x-70b-instruct",
    messages=[{"role": "user", "content": "สวัสดี"}]
)
print(chat_completion.choices[0].message.content)

สวัสดีครับ! ยินดีต้อนรับเข้าสู่การแชทของเรา หากคุณมีคำถามหรือต้องการความช่วยเหลือ กรุณาแจ้งให้เราทราบ เรายินดีที่จะช่วยคุณ.


In [116]:
# LLama Intregetion
# from llama_index.core.llms import ChatMessage, MessageRole
# from llama_index.llms.openai_like import OpenAILike

# llm = OpenAILike(model="typhoon-v1.5x-70b-instruct",
#                 api_base="https://api.opentyphoon.ai/v1",
#                 context_window=8192,
#                 is_chat_model=True,
#                 max_tokens=768,
#                 is_function_calling_model=False,
#                 api_key=os.getenv('API_KEY'))

# response = llm.chat([ChatMessage(role=MessageRole.USER, content='สวัสดี')])
# print(str(response))

### Labeling the Topic Category and Keywords

In [117]:
# Function call Typhoon API
def get_llm_values(text, system_message):
    try:
        chat_completion = client.chat.completions.create(
            model="typhoon-v1.5x-70b-instruct",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": text}
            ]
        )
        category = chat_completion.choices[0].message.content.strip()
        time.sleep(8)  # Avoid rate limit (10 requests/min)
        
        return category
        
    except Exception as e:
        print(f"Error categorizing text: {e}")
        return None

In [118]:
# Define topic category message
topic_category_message = (
    "Your task is to categorize given text into these categories.\n"
    "categories = { 'การจัดซื้อจัดหา', 'การผลิต', 'การจัดการสินค้าคงคลัง', "
    "'โลจิสติกส์และการกระจายสินค้า', 'การวางแผนและพยากรณ์ความต้องการ', 'การนำเข้า-ส่งออกและการค้าระหว่างประเทศ', "
    "'ความยั่งยืนและจริยธรรมในซัพพลายเชน' }\n"
    "If the text does not relate to any category on this list, answer 'อื่นๆ' \n"
    "Answer with only category name"
)

# การจัดซื้อจัดหา (Procurement)
# การผลิต (Manufacturing)
# การจัดการสินค้าคงคลัง (Inventory Management)
# โลจิสติกส์และการกระจายสินค้า (Logistics and Distribution)
# การวางแผนและพยากรณ์ความต้องการ (Demand Planning and Forecasting)
# ความยั่งยืนและจริยธรรมในซัพพลายเชน (Sustainability and Ethics)
# การนำเข้า-ส่งออกและการค้าระหว่างประเทศ (Import-Export and International Trade)

In [119]:
# Define message to extract keywords
keywords_message = (
    "Your task is extract keywords that contain in given abstract \n"
    "The total number of keywords should be between 3 to 5. \n"
    "Keywords can be in either English or Thai. \n"
    "If the text appears inside parentheses, it should not be considered as a keyword. \n"
    "Return the keywords as a concatenated string, separated by commas."
)

In [120]:
df = pd.read_csv('../data/abstract_table.csv')
df.head()

,project,abstract
0,การจัดทำคู่มือขั้นตอนการดำเนินการการนำเข้าคราฟ...,ผู้วิจัยได้ตระหนักถึงความยุ่งยากของขั้นตอนการน...
1,การเสนอแนวทางในการพัฒนาและสร้างความสัมพันธ์กับ...,งานวิจัยครั้งนี้มีวัตถุประสงค์เพื่อเสนอแนวทางใ...
2,การพัฒนามาตรฐานรถขนส่งวัตถุอันตรายที่เข้ามาในค...,ดำเนินธุรกิจเป็นผู้นำเข้า และจัดจำหน่ายสินค้าก...
3,แนวทางการปรับปรุงกระบวนการการส่งเอกสารใบกำกับภ...,การวิจัยครั้งนี้มีวัตถุประสงค์ เพื่อศึกษาขั้นต...
4,การศึกษาเทคนิคการพยากรณ์ยอดขายสายไฟที่เหมาะสม,จากสถานการณ์การแพร่ระบาดของเชื้อไวรัสโคโรนา 20...


In [122]:
# Add Label for Topic Category
start_time = time.time()

df['category'] = df['abstract'].apply(lambda x: get_llm_values(x, topic_category_message))

end_time = time.time()

# Calculate elapsed time in minutes
runtime_minutes = (end_time - start_time) / 60
print(f"Run time for label category: {runtime_minutes:.0f} minutes")

Run time for label category: 53 minutes


In [133]:
# Add Label for Topic Category
start_time = time.time()

df['keywords'] = df['abstract'].apply(lambda x: get_llm_values(x, keywords_message))
end_time = time.time()

# Calculate elapsed time in minutes
runtime_minutes = (end_time - start_time) / 60
print(f"Run time for get keywords: {runtime_minutes:.0f} minutes")

Run time for get keywords: 54 minutes


In [137]:
### Checking category in scope and add category index columns
topic_cat_list = ['การจัดซื้อจัดหา', 'การผลิต', 'การจัดการสินค้าคงคลัง', 
                  'โลจิสติกส์และการกระจายสินค้า', 'การวางแผนและพยากรณ์ความต้องการ', 
                  'การนำเข้า-ส่งออกและการค้าระหว่างประเทศ', 
                  'ความยั่งยืนและจริยธรรมในซัพพลายเชน','อื่นๆ']

In [140]:
df['category'].value_counts()

topic_category
การจัดการสินค้าคงคลัง                     138
โลจิสติกส์และการกระจายสินค้า               52
การนำเข้า-ส่งออกและการค้าระหว่างประเทศ     48
การจัดซื้อจัดหา                            47
การผลิต                                    32
การวางแผนและพยากรณ์ความต้องการ             20
อื่นๆ                                       2
ความยั่งยืนและจริยธรรมในซัพพลายเชน          1
Name: count, dtype: int64

In [141]:
category_mapping = {
    'การจัดซื้อจัดหา': 1,
    'การผลิต': 2,
    'การจัดการสินค้าคงคลัง' : 3,
    'โลจิสติกส์และการกระจายสินค้า': 4,
    'การวางแผนและพยากรณ์ความต้องการ': 5,
    'การนำเข้า-ส่งออกและการค้าระหว่างประเทศ': 6,
    'ความยั่งยืนและจริยธรรมในซัพพลายเชน': 7,
    'อื่นๆ': 8,
}

# Add a new 'topic_category_id' column based on the mapping
df['category_id'] = df['category'].map(category_mapping)

In [148]:
# Save to output CSV
df.to_csv('../data/abstract_label_added.csv', index=False)